In [1]:
import requests
import urllib
import urllib2
import re
import json
import pandas as pd
import xml.etree.ElementTree as ET

from collections import defaultdict
from bs4 import BeautifulSoup
from scrapy import Selector
from scrapy.selector import HtmlXPathSelector
from weather import Weather

In [2]:
search_text = "tumkur"

<h4>Yahoo Weather Information</h4>

In [3]:
try:
    weather = Weather()
    location = weather.lookup_by_location(search_text)
    condition = location.condition()
    yahoo_data_now = ("Yahoo", int((int(condition['temp'])-32)*float(0.5556)), "", "", "")

    yahoo_url = "https://query.yahooapis.com/v1/public/yql?"
    yql_query = "select * from weather.forecast where woeid in (select woeid from geo.places(1) where text='{}')".format(search_text)
    yql_url = yahoo_url + urllib.urlencode({'q':yql_query}) + "&format=json"
    result = urllib2.urlopen(yql_url).read()
    data = json.loads(result)

    week_data = data['query']['results']["channel"]["item"]["forecast"]
    day_1 = int((int(week_data[1]['high']) - 32) * 0.5556)
    day_2 = int((int(week_data[2]['high']) - 32) * 0.5556)
    day_3 = int((int(week_data[3]['high']) - 32) * 0.5556)
    day_4 = int((int(week_data[4]['high']) - 32) * 0.5556)
    day_5 = int((int(week_data[5]['high']) - 32) * 0.5556)
    yahoo_data = ("1.Yahoo", int((int(condition['temp'])-32)*float(0.5556)), day_1, day_2, day_3, day_4, day_5)
except:
    yahoo_data = (
        "1.Yahoo",
        "NA",
        "NA",
        "NA",
        "NA",
        "NA",
        "NA",
    )
print yahoo_data

('1.Yahoo', 25, 25, 27, 27, 27, 27)


<h4>The Weather Network</h4>

In [4]:
base_url = "https://www.theweathernetwork.com/in/api/location/search?searchText={}".format(search_text)
req = requests.get(base_url)

data = req.json()
data_url = []
if data:
    for i in data:
        if i["name"] == search_text.title() and i["country"] == "India":
            each_url = "https://www.theweathernetwork.com{}".format(i["url"])            
            data_url.append({"each_url": each_url, "code": i["code"]})

try:            
    if data_url:
    
        req_url = data_url[0]
        current_weather_url = "https://www.theweathernetwork.com/in/api/savedlocation/index/?placecodes={}".format(req_url["code"])
        current_weather_req = requests.get(current_weather_url).json()
        current_day_temp = current_weather_req[0]["temperature"]
    
        resp = urllib2.urlopen(req_url["each_url"]).read()
        soup = BeautifulSoup(resp, "lxml")
    
        current_data_day_1 = soup.find_all("div", attrs={"class": "day_1"})
        body = """{}""".format(current_data_day_1[0])
        selection_day_1 = Selector(text=body).xpath("//div").extract()
        temp_day_1 = re.findall("\d+", selection_day_1[5])[0]

        current_data_day_2 = soup.find_all("div", attrs={"class": "day_2"})
        body = """{}""".format(current_data_day_2[0])
        selection_day_2 = Selector(text=body).xpath("//div").extract()
        temp_day_2 = re.findall("\d+", selection_day_2[5])[0]

        current_data_day_3 = soup.find_all("div", attrs={"class": "day_3"})
        body = """{}""".format(current_data_day_3[0])
        selection_day_3 = Selector(text=body).xpath("//div").extract()
        temp_day_3 = re.findall("\d+", selection_day_3[5])[0]

        current_data_day_4 = soup.find_all("div", attrs={"class": "day_4"})
        body = """{}""".format(current_data_day_4[0])
        selection_day_4 = Selector(text=body).xpath("//div").extract()
        temp_day_4 = re.findall("\d+", selection_day_4[5])[0]

        current_data_day_5 = soup.find_all("div", attrs={"class": "day_5"})
        body = """{}""".format(current_data_day_5[0])
        selection_day_5 = Selector(text=body).xpath("//div").extract()
        temp_day_5 = re.findall("\d+", selection_day_5[5])[0]

        weather_network_data = (
            "2.Weather Network",
            current_day_temp,
            temp_day_1,
            temp_day_2,
            temp_day_3,
            temp_day_4,
            temp_day_5,
        )
    else:
        weather_network_data = (
            "2.Weather Network",
            "NA",
            "NA",
            "NA",
            "NA",
            "NA",
            "NA",
        )
except:
        weather_network_data = (
            "2.Weather Network",
            "NA",
            "NA",
            "NA",
            "NA",
            "NA",
            "NA",
        )
print weather_network_data  

('2.Weather Network', u'26', u'26', u'28', u'28', u'28', u'28')


<h4> MSN Weather </h4>

In [5]:
try:
    base_url = "http://weather.service.msn.com/data.aspx?weasearchstr={}&culture=en-US&weadegreetype=F&src=outlook".format(search_text)
    req = requests.get(base_url)
    root = ET.fromstring(req.content)
    data = root.__dict__['_children'][0].__dict__['_children'][0].__dict__['attrib']
    data_1 = root.__dict__['_children'][0].__dict__["_children"][3].__dict__['attrib']['high']
    data_2 = root.__dict__['_children'][0].__dict__["_children"][4].__dict__['attrib']['high']
    data_3 = root.__dict__['_children'][0].__dict__["_children"][5].__dict__['attrib']['high']
    msn_weather_data = (
        "3.MSN Weather",
        int((int(data['temperature'])-32)*float(0.5556)),
        int((int(data_1)-32)*float(0.5556)),
        int((int(data_2)-32)*float(0.5556)),
        int((int(data_3)-32)*float(0.5556)),
        "NA",
        "NA",
    )
except Exception as e:
    msn_weather_data = (
        "3.MSN Weather",
        "NA",
        "NA",
        "NA",
        "NA",
        "NA",
        "NA",
    )
print msn_weather_data  

('3.MSN Weather', 27, 28, 29, 28, 'NA', 'NA')


<h4> Accurate Weather </h4>

In [6]:
search_url = "https://www.accuweather.com/en/search-locations"
search_data = {"s": search_text.title()}
try:
    req = requests.post(search_url, search_data)
    resp = req.__dict__['_content']

    soup = BeautifulSoup(resp, "lxml")
    data_input_box = soup.find_all("input", attrs={'id': 's'})

    data_str = str(data_input_box[0])
    data_url = re.findall("(https://www.accuweather.com/\w+/\w+/\w+/[0-9]+/[\w-]+/[0-9]+)\"", data_str)[0]

    base_url = data_url.replace("weather-forecast", "current-weather")
    base_link_url = base_url.replace("current-weather", "daily-weather-forecast")
    base_links = ["{}?day={}".format(base_link_url, i) for i in range(1, 6)]

    req = urllib2.Request(base_url)
    read_url = urllib2.urlopen(req)
    response = read_url.read()
    soup = BeautifulSoup(response, "lxml")
    five_day_data = soup.find_all("li", attrs={'data-href': base_links})

    column_header = ('Day', 'Date', 'real_feel', 'precipitation', 'description')
    data = []
    for day in five_day_data:
        hxs = HtmlXPathSelector(day).xpath("/html/body/p/text()").extract()
        doc = """{}""".format(hxs[0].encode("utf-8"))
        each_day = re.sub("\n+",
                          ",",
                          doc.decode("unicode_escape").encode("ascii", "ignore").replace("/", "").replace("More", "")
                         )
        data.append(filter(None, tuple(each_day.split(",")))) 
    accurate_weather_data = ("4.Accurate Weather", data[0][2], data[1][2], data[3][2], data[4][2], "NA", "NA")
except:
    accurate_weather_data = (
        "4.Accurate Weather",
        "NA",
        "NA",
        "NA",
        "NA",
        "NA",
        "NA",
    )
print accurate_weather_data

('4.Accurate Weather', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA')


<h4>Weather Bug Weather</h4>

In [7]:
base_url = "https://weather.weatherbug.com/api/localities/search?query={}".format(search_text)
slug_data = requests.get(base_url).json()[0]['SlugName']
data_url = "https://weather.weatherbug.com/weather-forecast/10-day-weather/{}".format(slug_data)

data_req = urllib2.urlopen(data_url)
response = data_req.read()
soup = BeautifulSoup(response, "lxml")
day_data = soup.find_all("div", attrs={"class": "day-card__desktop"})
temp_value = []
for each in day_data:
    body = """{}""".format(each)
    body = body.decode("utf-8").encode('ascii','ignore').replace("\n", "").replace("\r", "")
    select = Selector(text=body).xpath('//div[@class="temp"]/text()').extract()
    temp = int(round((int(select[0].strip()) - 32) * float(0.5556)))
    temp_value.append(temp)
weather_bug_data = ("5.Weather Bug", temp_value[0], temp_value[1], temp_value[2], temp_value[3], temp_value[4], temp_value[5])

<h4>BBC Weather</h4>

In [8]:
search_url = "http://www.bbc.com/locator/default/en-GB/autocomplete.json?search={}&filter=international".format(search_text)

city_data = requests.get(search_url).json()
try:
    weather_data = []
    if city_data:
        city_data = city_data[0]
        city_id = city_data['id']
    
        weather_url = "http://www.bbc.com/weather/{}".format(city_id)
        data_req = urllib2.urlopen(weather_url)
        response = data_req.read()
        soup = BeautifulSoup(response, "lxml")
        day_data = soup.find_all("td", attrs={"class": "min-temp"})
        for each in day_data:
            body = """{}""".format(each)
            body = body.decode("utf-8").encode('ascii','ignore')
            select = Selector(text=body).xpath('//span[@data-unit="c"]/text()').extract()
            weather_data.append(int(select[0]))
        bbc_weather_data = ("6.BBC Weather", weather_data[0], weather_data[1], weather_data[2], weather_data[3], weather_data[4], "NA")
    else:
        bbc_weather_data = ("6.BBC Weather", "NA", "NA", "NA", "NA", "NA", "NA")
except Exception as e:
    print e.message
    bbc_weather_data = ("6.BBC Weather", "NA", "NA", "NA", "NA", "NA", "NA")
print bbc_weather_data

('6.BBC Weather', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA')


<h4>The Weather Channel</h4>

In [9]:
search_url = "https://dsx.weather.com/x/v2/web/loc/en_US/1/4/5/9/11/13/19/21/1000/1001/1003/us^/{}?format=json&pg=0%2C10".format(search_text)

try:
    resp = requests.get(search_url).json()
    data_url = "https://weather.com/en-IN/weather/tenday/l/{}".format(resp[0]['key'])

    data_req = urllib2.urlopen(data_url)
    response = data_req.read()
    soup = BeautifulSoup(response, "lxml")
    day_data = soup.find_all("td", attrs={"class": "temp"})
    weather_data = []
    for each in day_data:
        body = """{}""".format(each)
        body = body.decode("utf-8").encode('ascii','ignore')
        select = Selector(text=body).xpath('//div/span/text()').extract()
        weather_data.append(int(select[0]))
    weather_channel_data = ("7.The Weather Channel", weather_data[0], weather_data[1], weather_data[2], weather_data[3], weather_data[4], weather_data[5])
except:
    weather_channel_data = ("7.The Weather Channel", "NA", "NA", "NA", "NA", "NA", "NA")
print weather_channel_data

('7.The Weather Channel', 27, 27, 29, 28, 29, 30)


<h4>The Skymet Weather</h4>

In [10]:
search_url = "https://www.skymetweather.com/external/searchlocation?q={}&l=en&c=internal".format(search_text)

try:
    resp = requests.get(search_url).json()
    data_url = resp[0]['value']

    data_req = urllib2.urlopen(data_url)
    response = data_req.read()
    soup = BeautifulSoup(response, "lxml")
    day_data = soup.find_all("div", attrs={"class": "Forecast7Day"})
    body = """{}""".format(day_data[0])
    body = body.decode("utf-8").encode('ascii','ignore')
    select = Selector(text=body).xpath('//span[@class="maxt"]/span/span[@class="c"]/text()').extract()
    skymet_weather_data = ("8.The Skymet Weather", select[0], select[1], select[2], select[3], select[4], select[5])
except:
    skymet_weather_data = ("8.The Skymet Weather", "NA", "NA", "NA", "NA", "NA", "NA")
print skymet_weather_data

('8.The Skymet Weather', u'31', u'30', u'30', u'33', u'33', u'32')


<h2>Consolidated Weather Data</h2>

In [11]:
header = ("Sites", "Current Weather", "Day_1", "Day_2", "Day_3", "Day_4", "Day_5")
dataset = [
    yahoo_data,
    msn_weather_data,
    weather_network_data,
    accurate_weather_data,
    weather_bug_data,
    bbc_weather_data,
    weather_channel_data,
    skymet_weather_data,
]
df = pd.DataFrame(data=dataset, columns=header)
print search_text.title()+"\n"
print df.set_index("Sites")

Tumkur

                      Current Weather Day_1 Day_2 Day_3 Day_4 Day_5
Sites                                                              
1.Yahoo                            25    25    27    27    27    27
3.MSN Weather                      27    28    29    28    NA    NA
2.Weather Network                  26    26    28    28    28    28
4.Accurate Weather                 NA    NA    NA    NA    NA    NA
5.Weather Bug                      26    26    28    27    27    27
6.BBC Weather                      NA    NA    NA    NA    NA    NA
7.The Weather Channel              27    27    29    28    29    30
8.The Skymet Weather               31    30    30    33    33    32
